In [1]:
import dspy
import os
# from dotenv import load_dotenv
import requests
# load_dotenv()
from dspy.retrieve.chromadb_rm import ChromadbRM
import os
import openai
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb

os.environ["OPENAI_API_KEY"] = "sk-"

llm = dspy.OpenAI(
    model='gpt-3.5-turbo',
    api_key=os.environ['OPENAI_API_KEY'],
    max_tokens=2000
)


In [2]:
llm("hi")

['Hello! How can I assist you today?']

In [3]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/Users/derrickmwiti/.env/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

In [5]:
# trainset

In [10]:
import langchain 
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

DATA_PATH = "documents"
loader = DirectoryLoader(DATA_PATH,
                         glob='*.txt',
                         loader_cls=TextLoader)
documents = loader.load()
texts = text_splitter.split_documents(documents)

In [13]:
texts[0].page_content

"Dutch cuisine (Dutch: Nederlandse keuken) is formed from the cooking traditions and practices of the Netherlands. The country's cuisine is shaped by its location in the fertile North Sea river delta of the European Plain, giving rise to fishing, farming (for crops, dairy and meat) and overseas trade. The Burgundian-Habsburg court enriched the cuisine of the Dutch elite in the 15th and 16th century, so did the colonial spice trade in the 17th century."

In [14]:
documents = []
ids = []
for i, page in enumerate(texts):
    text = page.page_content
    documents.append(text)
    ids.append(str(i)) 
    
embedding_function = OpenAIEmbeddingFunction(
    api_key=os.environ['OPENAI_API_KEY'],
    model_name="text-embedding-ada-002"
)

chroma_client = chromadb.PersistentClient(path="./amsterdam_db")
 
collection = chroma_client.get_or_create_collection(name="amsterdam_collection", embedding_function=embedding_function)

collection.add(documents=documents,ids=ids)

retriever_model = ChromadbRM(
    collection_name = 'amsterdam_collection',
   persist_directory =  './amsterdam_db',
    embedding_function=embedding_function,
    k=5
)

dspy.settings.configure(lm=llm, rm=retriever_model)

results = retriever_model("How many kilometers of the earths surface does Europe cover?", k=5)

for result in results:
    print("Document:", result.long_text, "\n")

Document: Europe covers about 10.18 million km2 (3.93 million sq mi), or 2% of Earth's surface (6.8% of land area), making it the second-smallest continent (using the seven-continent model). Politically, Europe is divided into about fifty sovereign states, of which Russia is the largest and most populous, spanning 39% of the continent and comprising 15% of its population. Europe had a total population of about 745 million (about 10% of the world population) in 2021; the third-largest after Asia and Africa.[2][3] The European climate is largely affected by warm Atlantic currents that temper winters and 

Document: Europe makes up the western fifth of the Eurasian landmass.[28] It has a higher ratio of coast to landmass than any other continent or subcontinent.[237] Its maritime borders consist of the Arctic Ocean to the north, the Atlantic Ocean to the west and the Mediterranean, Black and Caspian Seas to the south.[238] 

Document: The population of Europe was about 742 million in 2023

In [15]:
context = collection.query(query_texts=['"Tell me about the Dutch cuisines'], n_results=3)
context = context['documents']
context

[["Dutch cuisine (Dutch: Nederlandse keuken) is formed from the cooking traditions and practices of the Netherlands. The country's cuisine is shaped by its location in the fertile North Sea river delta of the European Plain, giving rise to fishing, farming (for crops, dairy and meat) and overseas trade. The Burgundian-Habsburg court enriched the cuisine of the Dutch elite in the 15th and 16th century, so did the colonial spice trade in the 17th century.",
  'Southern Dutch cuisine constitutes the cuisine of the Dutch provinces of North Brabant and Limburg and the Flemish Region in Belgium. It is renowned for its many rich pastries, soups, stews and vegetable dishes and is often called Burgundian which is a Dutch idiom invoking the rich Burgundian court which ruled the Low Countries in the Middle Ages renowned for its splendor and great feasts.',
  'As the Dutch Republic entered its Golden Age in the 17th century, dishes of this kind became available to the wealthy middle class as well,

In [16]:
print(retriever_model('What is the population of Amsterdam?'))

[{'id': '254', 'score': 0.18351222574710846, 'long_text': 'excludes the relatively populous municipality of Zaanstad. The largest area by population, the Amsterdam Metropolitan Area (Dutch: Metropoolregio Amsterdam), has a population of 2,33\xa0million.[235] It includes for instance Zaanstad, Wormerland, Muiden, Abcoude, Haarlem, Almere and Lelystad but excludes Graft-De Rijp. Amsterdam is part of the conglomerate metropolitan area Randstad, with a total population of 6,659,300 inhabitants.[236]', 'metadatas': None}, {'id': '138', 'score': 0.1969064623117447, 'long_text': "In 1300, Amsterdam's population was around 1,000 people.[91] While many towns in Holland experienced population decline during the 15th and 16th centuries, Amsterdam's population grew,[92] mainly due to the rise of the profitable Baltic maritime trade especially in grain after the Burgundian victory in the Dutch–Hanseatic War in 1441.[93] The population of Amsterdam was only modest compared to the towns and cities of

In [19]:
class GenerateAnswer(dspy.Signature):
    """ Answer questions based on the provided context."""

    context = dspy.InputField(desc="Context for answering the question")
    question = dspy.InputField(desc="The question")
    answer = dspy.OutputField(desc="Answer")

In [20]:

r_model = dspy.Retrieve(k=3)

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = r_model
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [41]:
uncompiled_rag = RAG()
question = "Tell me about Coffee in Dutch"
rag(question)

Prediction(
    context=['Koffie verkeerd (literally "wrong coffee"), or Café au lait, consists of equal parts black coffee and hot milk. The Dutch drink tea without milk and the tea is quite a lot weaker than typical English or Irish types of tea which are stronger and are usually taken with milk. In Dutch bars, tea with freshly chopped ginger (verse gemberthee) or with fresh mint leaves (verse muntthee) or very popular. In the autumn and winter hot chocolate or chocomel is drunk. Two other traditional Dutch drinks, anijsmelk (hot milk with aniseed) and kwast (hot water with lemon juice)[26] are hardly drunk any', "Dutch people invite friends over for koffietijd (coffee time), which consists of coffee and cake or biscuits, served between 10:00 and 11:00 am (before lunch), 4:00\xa0pm (between lunch and dinner) or between 7:00\xa0pm and 8:00\xa0pm (after dinner). Dutch thrift of the 1940's and 1950's, when the country was rebuilding the destruction of World War II, led to the famous sta

In [22]:
rag(question).answer

'Coffee in Dutch culture is enjoyed in various forms, such as "koffie verkeerd" which is a mix of black coffee and hot milk. "Koffietijd" is a social tradition where friends gather for coffee and cake or biscuits at specific times during the day.'

In [30]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=False, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(rag, metric=metric)

Average Metric: 8 / 50  (16.0%)


/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' '✔️ [True]' 'False' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",The information provided does not specify the river near the Crichton Collegiate Church.,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,✔️ [True]


16.0

In [23]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(rag, trainset=trainset)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:46<00:00,  2.34s/it]

Bootstrapped 0 full traces after 20 examples in round 0.


In [24]:
# Ask any question you like to this simple RAG program.
my_question = "What are some dinner options in Dutch cuisine?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: What are some dinner options in Dutch cuisine?
Predicted Answer: AVG, asparagus with ham, egg, and potatoes, satay with chips, babi panggang speciaal, broodje bakkeljauw, Biefstuk, Varkenshaas, Ossenhaas, etc.
Retrieved Contexts (truncated): ['Dinner, traditionally served early by international standards, starts around or even before 6 p.m. It is not uncommon for restaurants outside of major cities to be closed by 8 p.m.[citation needed] Th...', 'A seasonal dish, asparagus is popularly eaten with ham, egg, and potatoes, with a butter sauce.\n A simple Dutch meal traditionally consists of meat, potatoes, butter, a boiled vegetable, and salad.\n S...', 'It is the only Dutch culinary region which developed an haute cuisine and it forms the base of most traditional Dutch restaurants including typical main courses served such as Biefstuk, Varkenshaas, O...']


In [26]:
llm.inspect_history(n=1)





Answer questions based on the provided context.

---

Question: Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?
Answer: Tae Kwon Do Times

Question: This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?
Answer: The Waltz King

Question: On the coast of what ocean is the birthplace of Diogal Sakho?
Answer: Atlantic

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas founded in what year?
Answer: 2010

Question: Which company distributed this 1977 American animated film produced by Walt Disney Productions for which Sherman Brothers wrote songs?
Answer: Buena Vista Distribution

Question: Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their fi

In [27]:
for name, parameter in compiled_rag.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

generate_answer
Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'})



In [28]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=False, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

Average Metric: 15 / 50  (30.0%)


/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' '✔️ [True]' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The National Hockey League (NHL) conducted the draft.,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",Don River,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...","The English king during the 10th century A.D. was Edward the Elder, who was the son of King Alfred the Great and Ealhswith.",False


30.0

## Define Custom Metric

In [35]:
# Reference - https://github.com/stanfordnlp/dspy/blob/main/examples/tweets/tweet_metric.py

metricLM = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=1000, model_type='chat')

# Signature for LLM assessments.

class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""
    
    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer
    question = gold.question
    
    print(f"Test Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    
    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    overall = f"Please rate how well this answer answers the question, `{question}` based on the context.\n `{predicted_answer}`"
    
    with dspy.context(lm=metricLM):
        context = dspy.Retrieve(k=5)(question).passages
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context=context, assessed_question=overall, assessed_answer=predicted_answer)
    
    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")
    
    
    total = float(detail.assessment_answer) + float(faithful.assessment_answer)*2 + float(overall.assessment_answer)
    
    return total / 5.0

## Inspect the metric

In [36]:
test_example = dspy.Example(question="Tell me about Coffee in Dutch")
test_pred = dspy.Example(answer="Coffee in Dutch culture is enjoyed in various forms, such as 'koffie verkeerd' which is a mix of black coffee and hot milk. 'Koffietijd' is a social tradition where friends gather for coffee and cake or biscuits at specific times during the day")

type(llm_metric(test_example, test_pred))

Test Question: Tell me about Coffee in Dutch
Predicted Answer: Coffee in Dutch culture is enjoyed in various forms, such as 'koffie verkeerd' which is a mix of black coffee and hot milk. 'Koffietijd' is a social tradition where friends gather for coffee and cake or biscuits at specific times during the day
Faithful: 5
Detail: 4
Overall: 5


float

In [37]:
metricLM.inspect_history(n=3)





Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question.

Assessed Question: The evaluation criterion.

Assessed Answer: The answer to the question.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context:
[1] «Koffie verkeerd (literally "wrong coffee"), or Café au lait, consists of equal parts black coffee and hot milk. The Dutch drink tea without milk and the tea is quite a lot weaker than typical English or Irish types of tea which are stronger and are usually taken with milk. In Dutch bars, tea with freshly chopped ginger (verse gemberthee) or with fresh mint leaves (verse muntthee) or very popular. In the autumn and winter hot chocolate or chocomel is drunk. Two other traditional Dutch drinks, anijsmelk (hot milk with aniseed) and kwast (hot water with lemon juice)[26] a

## Evaluate our RAG Program before it is compiled

In [38]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

evaluate(RAG(), metric=llm_metric)


  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: No



Average Metric: 1.8 / 1  (180.0):   2%|██▋                                                                                                                                   | 1/50 [00:06<05:08,  6.30s/it]

Faithful: 1
Detail: 2
Overall: 5
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.



Average Metric: 3.4000000000000004 / 2  (170.0):   2%|██▍                                                                                                                    | 1/50 [00:12<05:08,  6.30s/it]
Average Metric: 3.4000000000000004 / 2  (170.0):   4%|████▊                                                                                                                  | 2/50 [00:12<04:57,  6.20s/it]

...


Average Metric: 91.59999999999998 / 50  (183.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:31<00:00,  5.42s/it]

Faithful: 5
Detail: 1
Overall: 5
Average Metric: 91.59999999999998 / 50  (183.2%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1.8' '1.6' '0.8' '2.2' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",No,1.8
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.,1.6
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,0.8
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",The information provided does not specify the river near the Crichton Collegiate Church.,2.2
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,0.8


183.2

In [39]:
metricLM.inspect_history(n=3)





Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question.

Assessed Question: The evaluation criterion.

Assessed Answer: The answer to the question.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context:
[1] «On 18 September 2017, it was announced by Eberhard van der Laan in an open letter to Amsterdam citizens that Kajsa Ollongren would take up his office as acting Mayor of Amsterdam with immediate effect due to ill health.[233] Ollongren was succeeded as acting Mayor by Eric van der Burg on 26 October 2017 and by Jozias van Aartsen on 4 December 2017.»
[2] «from the late 16th century, when city and province became part of the Republic of the Seven United Netherlands. Last came the city's official motto: Heldhaftig, Vastberaden, Barmhartig ("Heroic, Determined, Merciful"),

## BootstrapFewShot

In [42]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=llm_metric, max_labeled_demos=8, max_rounds=3)

# also common to init here, e.g. Rag()
compiled_rag = teleprompter.compile(uncompiled_rag, trainset=trainset)


  0%|                                                                                                                                                                                | 0/20 [00:00<?, ?it/s]

Test Question: At My Window was released by which American singer-songwriter?
Predicted Answer: N/A



  5%|████████▍                                                                                                                                                               | 1/20 [00:07<02:22,  7.53s/it]

Faithful: 5
Detail: 4
Overall: N/A
Failed to run or to evaluate example Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}) with <function llm_metric at 0x30a6ef740> due to could not convert string to float: 'N/A'.
Test Question: which  American actor was Candace Kita  guest starred with 
Predicted Answer: The answer cannot be determined based on the provided context.



 10%|████████████████▊                                                                                                                                                       | 2/20 [00:13<02:01,  6.78s/it]

...

Faithful: 1
Detail: 1
Overall: 5
Test Question: In what year was the club founded that played Manchester City in the 1972 FA Charity Shield
Predicted Answer: AFC Ajax was founded in 1900.



 30%|██████████████████████████████████████████████████▍                                                                                                                     | 6/20 [00:41<01:36,  6.89s/it]


Faithful: 1
Detail: 1
Overall: 5



  0%|                                                                                                                                                                                | 0/20 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                | 0/20 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 1 examples in round 2.


## Inspect the compiled prompt

In [43]:
compiled_rag("Tell me about the early history of Europe?").answer

'The early history of Europe includes ancient Greece and Rome, the fall of the Western Roman Empire, the Middle Ages, the Renaissance, and the Age of Discovery.'

In [44]:
llm.inspect_history(n=1)






Answer questions based on the provided context.

---

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?
Answer: Rosario Dawson

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

---

Follow the following format.

Context: Context for answering the question

Question: The question

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Answer

---

Context:
[1] «was the young Jewish girl Anne Frank, who died in the Bergen-Belsen concentration camp.[60] At the end of the Second World War, communication with the rest of the country

## Evaluate the Compiled RAG Program

In [45]:
evaluate(compiled_rag, metric=llm_metric)


  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: Insufficient information to determine if both Cangzhou and Qionghai are in the Hebei province of China.



Average Metric: 2.0 / 1  (200.0):   2%|██▋                                                                                                                                   | 1/50 [00:06<05:42,  6.99s/it]

Faithful: 2
Detail: 1
Overall: 5
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: National Hockey League (NHL)



Average Metric: 3.6 / 2  (180.0):   4%|█████▎                                                                                                                                | 2/50 [00:15<06:16,  7.84s/it]

...


Average Metric: 81.79999999999997 / 50  (163.6): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:46<00:00,  5.73s/it]

Faithful: 5
Detail: 1
Overall: 5
Average Metric: 81.79999999999997 / 50  (163.6%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2.0' '1.6' '0.8' '0.8' '1.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",Insufficient information to determine if both Cangzhou and Qionghai are in the Hebei province of China.,2.0
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,National Hockey League (NHL),1.6
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,0.8
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",Don River,0.8
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",Ealhswith had a son called Æthelweard by King Alfred the Great.,1.8


163.6

## BootstrapFewShotWithRandomSearch

In [46]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

teleprompter = BootstrapFewShotWithRandomSearch(metric=llm_metric, 
                                                max_bootstrapped_demos=4,
                                                max_labeled_demos=4, 
                                                max_rounds=1,
                                                num_candidate_programs=2,
                                                num_threads=2)

# also common to init here, e.g. Rag()
second_compiled_rag = teleprompter.compile(uncompiled_rag, trainset=trainset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 2 candidate sets.



  0%|                                                                                                                                                                                | 0/20 [00:00<?, ?it/s]

Test Question: At My Window was released by which American singer-songwriter?
Predicted Answer: Bob Dylan
Test Question: which  American actor was Candace Kita  guest starred with 
Predicted Answer: The question does not provide enough context to determine which American actor Candace Kita guest starred with.



Average Metric: 1.6 / 1  (160.0):   5%|██████▋                                                                                                                               | 1/20 [00:06<02:07,  6.72s/it]

Faithful: 1
Detail: 1
Overall: 5


...

Faithful: 1
Detail: 1
Overall: 5



Average Metric: 31.200000000000014 / 19  (164.2):  90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▎           | 18/20 [00:36<00:04,  2.18s/it]
Average Metric: 31.200000000000014 / 19  (164.2):  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▏     | 19/20 [00:36<00:02,  2.13s/it]

Test Question: This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?
Predicted Answer: Nita Strauss, ancestor of Johann Strauss
Faithful: 3
Detail: 2
Overall: 5



Average Metric: 32.000000000000014 / 20  (160.0):  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▏     | 19/20 [00:37<00:02,  2.13s/it]
Average Metric: 32.000000000000014 / 20  (160.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:37<00:00,  1.87s/it]

Faithful: 1
Detail: 1
Overall: 1
Average Metric: 32.000000000000014 / 20  (160.0%)
Score: 160.0 for set: [4]
Scores so far: [171.0, 163.0, 174.0, 160.0, 160.0]
Best score: 174.0
Average of max per entry across top 1 scores: 1.7400000000000002
Average of max per entry across top 2 scores: 2.0900000000000003
Average of max per entry across top 3 scores: 2.12
Average of max per entry across top 5 scores: 2.33
Average of max per entry across top 8 scores: 2.33
Average of max per entry across top 9999 scores: 2.33
5 candidate programs found.


In [47]:
second_compiled_rag("Which languages are spoken in Europe?").answer

'Russian, French, Italian, German, English, Romance languages, Germanic languages, Slavic languages, Uralic languages, Turkic languages, Basque, Semitic languages, languages of the Caucasus, African languages, Asian languages'

In [48]:
llm.inspect_history(n=1)





Answer questions based on the provided context.

---

Follow the following format.

Context: Context for answering the question

Question: The question

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Answer

---

Context:
[1] «Amsterdam's musical culture includes a large collection of songs that treat the city nostalgically and lovingly. The 1949 song "Aan de Amsterdamse grachten" ("On the canals of Amsterdam") was performed and recorded by many artists, including John Kraaijkamp Sr.; the best-known version is probably that by Wim Sonneveld (1962). In the 1950s Johnny Jordaan rose to fame with "Geef mij maar Amsterdam" ("I prefer Amsterdam"), which praises the city above all others (explicitly Paris); Jordaan sang especially about his own neighbourhood, the Jordaan ("Bij ons in de Jordaan"). Colleagues and»
[2] «sang especially about his own neighbourhood, the Jordaan ("Bij ons in de Jordaan"). Colleagues and contemporaries of Johnny include 

In [49]:
evaluate(second_compiled_rag, metric=llm_metric)


  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: The information to determine if both Cangzhou and Qionghai are in the Hebei province of China is not provided in the context.



Average Metric: 0.8 / 1  (80.0):   2%|██▋                                                                                                                                    | 1/50 [00:06<05:30,  6.74s/it]

Faithful: 1
Detail: 1
Overall: 1
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: The answer cannot be determined from the provided context.



Average Metric: 2.0 / 2  (100.0):   4%|█████▎                                                                                                                                | 2/50 [00:12<04:58,  6.23s/it]

...


Average Metric: 93.19999999999999 / 50  (186.4): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:22<00:00,  5.25s/it]

Faithful: 2
Detail: 1
Overall: 5
Average Metric: 93.19999999999999 / 50  (186.4%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.8' '1.2' '2.4' '0.8' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric,answer
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",The information to determine if both Cangzhou and Qionghai are in the Hebei province of China is not provided in the context.,0.8,nan
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The answer cannot be determined from the provided context.,1.2,nan
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",The answer cannot be determined from the provided context.,2.4,nan
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",Don River,0.8,nan
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,0.8,nan


186.4

## BayesianSignatureOptimizer


In [50]:
from dspy.teleprompt import BayesianSignatureOptimizer

llm_prompter = dspy.OpenAI(model='gpt-4', max_tokens=2000, model_type='chat')

teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=llm_metric,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=False)

kwargs = dict(num_threads=1, display_progress=True, display_table=0)
third_compiled_rag = teleprompter.compile(RAG(), devset=devset,
                                         optuna_trials_num=3,
                                         max_bootstrapped_demos=4,
                                         max_labeled_demos=4,
                                         eval_kwargs=kwargs)

[WARNING] BayesianSignatureOptimizer has been deprecated and replaced with MIPRO.  BayesianSignatureOptimizer will be removed in a future release. 

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 50 examples in dev set * 3 trials * # of LM calls in your program = (150 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 5 * 1 lm calls in program = 15 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own cal

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay Lightning of the National Hockey League (NHL)?
Predicted Answer: Steve Yzerman


  2%|███▎                                                                                                                                                                    | 1/50 [00:02<01:48,  2.21s/it]

Faithful: 1
Detail: 1
Overall: 1
Test Question: What is the name of this region of Italy, referring to the medieval March of Ancona and nearby marches of Camerino and Fermo, where the comune Pollenza is located?
Predicted Answer: Marche


...

Faithful: 1
Detail: 1
Overall: 5
Test Question:  Suzana S. Drobnjaković Ponti acted in a film loosely based on a book by who?
Predicted Answer: Unfortunately, based on the context provided, we cannot determine the answer to this question.


Average Metric: 88.39999999999998 / 49  (180.4):  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 49/50 [00:45<00:00,  1.02it/s]

Faithful: 4
Detail: 1
Overall: 5
Test Question: In what city was the Election Law Journal founded?
Predicted Answer: Amsterdam


Average Metric: 91.59999999999998 / 50  (183.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:46<00:00,  1.07it/s]
[I 2024-07-03 23:37:22,928] Trial 2 finished with value: 183.2 and parameters: {'13076461008_predictor_instruction': 0, '13076461008_predictor_demos': 0}. Best is trial 2 with value: 183.2.


Faithful: 5
Detail: 1
Overall: 5
Average Metric: 91.59999999999998 / 50  (183.2%)
Returning generate_answer = ChainOfThought(GenerateAnswer(context, question -> answer
    instructions=' Answer questions based on the provided context.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'Context for answering the question', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': 'The question', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'Answer', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)) from continue_program


In [51]:
third_compiled_rag("Tell me about the economy of Europe?")

Prediction(
    context=['By the millennium change, the EU dominated the economy of Europe, comprising the five largest European economies of the time: Germany, the United Kingdom, France, Italy, and Spain. In 1999, 12 of the 15 members of the EU joined the Eurozone, replacing their national currencies by the euro.', "The European Union, a political entity composed of 27 European states, comprises the largest single economic area in the world. Nineteen EU countries share the euro as a common currency.\nFive European countries rank in the top ten of the world's largest national economies in GDP (PPP). This includes (ranks according to the CIA): Germany (6), Russia (7), the United Kingdom (10), France (11) and Italy (13).[254]", "As a continent, the economy of Europe is currently the largest on Earth and it is the richest region as measured by assets under management with over $32.7\xa0trillion compared to North America's $27.1\xa0trillion in 2008.[251] In 2009 Europe remained the wealth

In [52]:
llm.inspect_history(n=1)





Answer questions based on the provided context.

---

Follow the following format.

Context: Context for answering the question

Question: The question

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Answer

---

Context:
[1] «By the millennium change, the EU dominated the economy of Europe, comprising the five largest European economies of the time: Germany, the United Kingdom, France, Italy, and Spain. In 1999, 12 of the 15 members of the EU joined the Eurozone, replacing their national currencies by the euro.»
[2] «The European Union, a political entity composed of 27 European states, comprises the largest single economic area in the world. Nineteen EU countries share the euro as a common currency.
Five European countries rank in the top ten of the world's largest national economies in GDP (PPP). This includes (ranks according to the CIA): Germany (6), Russia (7), the United Kingdom (10), France (11) and Italy (13).[254]»
[3] «As a contine

In [53]:
evaluate(third_compiled_rag, metric=llm_metric)

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: No


Average Metric: 1.8 / 1  (180.0):   2%|██▋                                                                                                                                   | 1/50 [00:00<00:46,  1.06it/s]

Faithful: 1
Detail: 2
Overall: 5
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.


Average Metric: 3.4000000000000004 / 2  (170.0):   4%|████▊                                                                                                                  | 2/50 [00:03<01:36,  2.00s/it]

...

Average Metric: 91.59999999999998 / 50  (183.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:48<00:00,  1.02it/s]

Faithful: 5
Detail: 1
Overall: 5
Average Metric: 91.59999999999998 / 50  (183.2%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1.8' '1.6' '0.8' '2.2' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",No,1.8
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.,1.6
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,0.8
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",The information provided does not specify the river near the Crichton Collegiate Church.,2.2
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,0.8


183.2

## Test Set Eval

In [55]:
# Evaluate Uncompiled
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

In [56]:
evaluate(uncompiled_rag, metric=llm_metric)

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: No


Average Metric: 1.8 / 1  (180.0):   2%|██▋                                                                                                                                   | 1/50 [00:02<01:40,  2.04s/it]

Faithful: 1
Detail: 2
Overall: 5
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.


Average Metric: 3.4000000000000004 / 2  (170.0):   4%|████▊                                                                                                                  | 2/50 [00:02<01:05,  1.36s/it]

...

Average Metric: 92.59999999999998 / 50  (185.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:52<00:00,  1.06s/it]

Faithful: 5
Detail: 1
Overall: 5
Average Metric: 92.59999999999998 / 50  (185.2%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1.8' '1.6' '0.8' '2.2' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",No,1.8
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.,1.6
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,0.8
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",The information provided does not specify the river near the Crichton Collegiate Church.,2.2
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,0.8


185.2

In [57]:
evaluate(compiled_rag, metric=llm_metric)

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: Insufficient information to determine if both Cangzhou and Qionghai are in the Hebei province of China.


Average Metric: 2.0 / 1  (200.0):   2%|██▋                                                                                                                                   | 1/50 [00:00<00:44,  1.11it/s]

Faithful: 2
Detail: 1
Overall: 5
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: National Hockey League (NHL)


Average Metric: 3.6 / 2  (180.0):   4%|█████▎                                                                                                                                | 2/50 [00:01<00:45,  1.05it/s]

...

Average Metric: 81.79999999999997 / 50  (163.6): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:45<00:00,  1.10it/s]

Faithful: 5
Detail: 1
Overall: 5
Average Metric: 81.79999999999997 / 50  (163.6%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2.0' '1.6' '0.8' '0.8' '1.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",Insufficient information to determine if both Cangzhou and Qionghai are in the Hebei province of China.,2.0
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,National Hockey League (NHL),1.6
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,0.8
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",Don River,0.8
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",Ealhswith had a son called Æthelweard by King Alfred the Great.,1.8


163.6

In [58]:
evaluate(second_compiled_rag, metric=llm_metric)

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: The information to determine if both Cangzhou and Qionghai are in the Hebei province of China is not provided in the context.


Average Metric: 0.8 / 1  (80.0):   2%|██▋                                                                                                                                    | 1/50 [00:00<00:41,  1.19it/s]

Faithful: 1
Detail: 1
Overall: 1
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: The answer cannot be determined from the provided context.


Average Metric: 2.0 / 2  (100.0):   4%|█████▎                                                                                                                                | 2/50 [00:01<00:43,  1.11it/s]

...

Average Metric: 93.19999999999999 / 50  (186.4): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:50<00:00,  1.01s/it]

Faithful: 2
Detail: 1
Overall: 5
Average Metric: 93.19999999999999 / 50  (186.4%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.8' '1.2' '2.4' '0.8' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric,answer
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",The information to determine if both Cangzhou and Qionghai are in the Hebei province of China is not provided in the context.,0.8,nan
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The answer cannot be determined from the provided context.,1.2,nan
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",The answer cannot be determined from the provided context.,2.4,nan
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",Don River,0.8,nan
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,0.8,nan


186.4

In [59]:
evaluate(third_compiled_rag, metric=llm_metric)


  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Test Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: No


Average Metric: 1.8 / 1  (180.0):   2%|██▋                                                                                                                                   | 1/50 [00:01<00:49,  1.00s/it]

Faithful: 1
Detail: 2
Overall: 5
Test Question: Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?
Predicted Answer: The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.


Average Metric: 3.4000000000000004 / 2  (170.0):   4%|████▊                                                                                                                  | 2/50 [00:01<00:44,  1.08it/s]

...

Average Metric: 91.59999999999998 / 50  (183.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:47<00:00,  1.06it/s]

Faithful: 5
Detail: 1
Overall: 5
Average Metric: 91.59999999999998 / 50  (183.2%)



/Users/derrickmwiti/.env/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1.8' '1.6' '0.8' '2.2' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,llm_metric
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Table\n* indicates ""Cities of COUNTRY or TERRITORY"" links.\nCountry (or territory)\tCapital\tPopulation\t% of country\tSource\n China\u202f*\tBeijing\t21,542,000\t1.5%\t[1] 2018\n Japan\u202f*\tTokyo\t14,094,034\t11.3%\t[2] 2023\n Russia\u202f*\tMoscow\t13,104,177\t9.0%\t[3] 2023\n DR Congo\u202f*\tKinshasa\t12,691,000\t13.2%\t[4] 2017\n Indonesia\u202f*\tJakarta\t10,562,088\t3.9%\t[5] 2020\n Egypt\u202f*\tCairo\t10,107,125\t9.3%\t[6] 2022\n...",No,1.8
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['Amsterdam holds two American football franchises: the Amsterdam Crusaders and the Amsterdam Panthers. The Amsterdam Pirates baseball team competes in the Dutch Major League. There...,The National Hockey League (NHL) conducted the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season.,1.6
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['city was finally liberated by Canadian forces on 5 May 1945, shortly before the end of the war in Europe.', 'In parallel, the Eastern Question...",Steve Yzerman,0.8
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['By the mid-19th century, there were three main conventions, one following the Don, the Volga–Don Canal and the Volga, the other following the Kuma–Manych Depression...",The information provided does not specify the river near the Crichton Collegiate Church.,2.2
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['During the Dark Ages, the Western Roman Empire fell under the control of various tribes. The Germanic and Slav tribes established their domains over Western...",King Alfred the Great,0.8


183.2